In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats

# Configure plotting defaults
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [3]:
# File paths 
path_nh_citations = 'data/final-csv/nh_citations.csv'
path_nh_ownership = 'data/final-csv/nh_ownership.csv'
path_nh_quality_mds = 'data/final-csv/nh_quality_mds.csv'
path_nh_survey = 'data/final-csv/nh_survey.csv'
path_pbj_non_nurse = 'data/final-csv/pbj_non_nurse.csv'
path_pbj_nurse = 'data/final-csv/pbj_nurse.csv'
path_qrp_provider = 'data/final-csv/qrp_provider.csv'

# Read CSVs into DataFrames with low_memory=False
df_nh_citations = pd.read_csv(path_nh_citations, low_memory=False)
df_nh_ownership = pd.read_csv(path_nh_ownership, low_memory=False)
df_nh_quality_mds = pd.read_csv(path_nh_quality_mds, low_memory=False)
df_nh_survey = pd.read_csv(path_nh_survey, low_memory=False)
df_pbj_non_nurse = pd.read_csv(path_pbj_non_nurse, low_memory=False)
df_pbj_nurse = pd.read_csv(path_pbj_nurse, low_memory=False)
df_qrp_provider = pd.read_csv(path_qrp_provider, low_memory=False)

In [6]:
# Ensure workdate is datetime
df_pbj_nurse['workdate'] = pd.to_datetime(df_pbj_nurse['workdate'])
df_pbj_non_nurse['workdate'] = pd.to_datetime(df_pbj_non_nurse['workdate'])

# 1. Daily Temporary Staffing Ratios for Each Role
# Nursing Roles
daily_ratios = pd.DataFrame()

# RN Daily Ratio
daily_ratios['rn_daily_ratio'] = (
    df_pbj_nurse['hrs_rn_ctr'] / 
    (df_pbj_nurse['hrs_rn_emp'] + df_pbj_nurse['hrs_rn_ctr'] + 1e-6)
)

# LPN Daily Ratio
daily_ratios['lpn_daily_ratio'] = (
    df_pbj_nurse['hrs_lpn_ctr'] / 
    (df_pbj_nurse['hrs_lpn_emp'] + df_pbj_nurse['hrs_lpn_ctr'] + 1e-6)
)

# CNA Daily Ratio
daily_ratios['cna_daily_ratio'] = (
    df_pbj_nurse['hrs_cna_ctr'] / 
    (df_pbj_nurse['hrs_cna_emp'] + df_pbj_nurse['hrs_cna_ctr'] + 1e-6)
)

# Support Roles (Non-Nursing) Daily Ratio
contract_cols = [col for col in df_pbj_non_nurse.columns if col.endswith('_ctr')]
employee_cols = [col for col in df_pbj_non_nurse.columns if col.endswith('_emp')]

daily_ratios['support_daily_ratio'] = (
    df_pbj_non_nurse[contract_cols].sum(axis=1) /
    (df_pbj_non_nurse[contract_cols].sum(axis=1) + df_pbj_non_nurse[employee_cols].sum(axis=1) + 1e-6)
)

# Add workdate and facility identifier
daily_ratios['workdate'] = df_pbj_nurse['workdate']
daily_ratios['provnum'] = df_pbj_nurse['provnum']

# 2. Variation Metrics
variation_metrics = daily_ratios.groupby('provnum').agg({
    'rn_daily_ratio': ['mean', 'std'],
    'lpn_daily_ratio': ['mean', 'std'],
    'cna_daily_ratio': ['mean', 'std'],
    'support_daily_ratio': ['mean', 'std']
}).round(4)

daily_facility_ratios = daily_ratios.set_index('workdate')
rolling_means = daily_facility_ratios.groupby('provnum').rolling(window=7, min_periods=1).mean()

# 3. Gap Identification (Option 2: Using vectorized operations)
k = 2  # Standard deviations above mean to identify gaps

gap_indicators = pd.DataFrame()
roles = ['rn_daily_ratio', 'lpn_daily_ratio', 'cna_daily_ratio', 'support_daily_ratio']

for role in roles:
    # Compute facility-specific mean and std for each row
    facility_mean = daily_ratios.groupby('provnum')[role].transform('mean')
    facility_std = daily_ratios.groupby('provnum')[role].transform('std')
    
    # Compute threshold and identify gaps
    threshold = facility_mean + k * facility_std
    gap_indicators[f'{role}_gap'] = (daily_ratios[role] > threshold).astype(int)

# Add dates and facility identifiers to gap indicators
gap_indicators['workdate'] = daily_ratios['workdate']
gap_indicators['provnum'] = daily_ratios['provnum']

# Summary statistics
print("\nGap Summary (percentage of days with staffing gaps by role):")
gap_summary = gap_indicators.iloc[:, :-2].mean() * 100
print(gap_summary)

# Time series patterns
daily_ratios['day_of_week'] = daily_ratios['workdate'].dt.day_name()
dow_patterns = daily_ratios.groupby('day_of_week')[
    ['rn_daily_ratio', 'lpn_daily_ratio', 'cna_daily_ratio', 'support_daily_ratio']
].mean()
print("\nDay-of-Week Patterns (average ratios):")
print(dow_patterns)



Gap Summary (percentage of days with staffing gaps by role):
rn_daily_ratio_gap         1.669931
lpn_daily_ratio_gap        1.884068
cna_daily_ratio_gap        1.744856
support_daily_ratio_gap    2.657011
dtype: float64

Day-of-Week Patterns (average ratios):
             rn_daily_ratio  lpn_daily_ratio  cna_daily_ratio  \
day_of_week                                                     
Friday             0.066302         0.088464         0.067255   
Monday             0.064209         0.084058         0.064742   
Saturday           0.074450         0.096512         0.074838   
Sunday             0.075093         0.096785         0.076156   
Thursday           0.058981         0.080271         0.059126   
Tuesday            0.058042         0.077725         0.058426   
Wednesday          0.058999         0.077213         0.056746   

             support_daily_ratio  
day_of_week                       
Friday                  0.258638  
Monday                  0.268647  
Saturday     

In [8]:
mdscensus_range = df_pbj_non_nurse['mdscensus'].max() - df_pbj_non_nurse['mdscensus'].min()
print("Range of mdscensus:", mdscensus_range)


Range of mdscensus: 742


In [9]:
# Calculate facility size (average mdscensus by facility)
facility_size = df_pbj_nurse.groupby('provnum')['mdscensus'].mean()
print("\nFacility Size Stats:")
print(f"Number of facilities: {len(facility_size)}")
print(f"Census range: {facility_size.min():.1f} to {facility_size.max():.1f}")

# Create size categories using quartile-based binning
# This divides the facilities into 4 equal groups
size_categories = pd.qcut(facility_size, q=4, labels=['Small', 'Medium', 'Large', 'Very-Large'])

print("\nFacility Size Categories (Quartile Based):")
print(size_categories.value_counts())

# 2. Calculate RN contract metrics
contract_metrics = df_pbj_nurse.groupby('provnum').agg({
    'hrs_rn_ctr': 'sum',
    'hrs_rn_emp': 'sum',
    'mdscensus': 'mean'
}).reset_index()

contract_metrics['rn_contract_ratio'] = (
    contract_metrics['hrs_rn_ctr'] / 
    (contract_metrics['hrs_rn_emp'] + contract_metrics['hrs_rn_ctr'] + 1e-6)
)

print("\nContract Metrics Stats:")
print(f"Number of facilities with contract metrics: {len(contract_metrics)}")
print(f"RN contract ratio range: {contract_metrics['rn_contract_ratio'].min():.3f} to {contract_metrics['rn_contract_ratio'].max():.3f}")

# 3. Create contract categories
contract_bins = [0, 0.05, 0.15, 1.0]
contract_labels = ['Low Contract', 'Medium Contract', 'High Contract']
contract_metrics['contract_category'] = pd.cut(
    contract_metrics['rn_contract_ratio'],
    bins=contract_bins,
    labels=contract_labels
)

# Add size categories
contract_metrics['size_category'] = size_categories.reindex(contract_metrics['provnum']).values

# 4. Merge with ownership data
# First, ensure provnum matches CCN format
ownership_clean = df_nh_ownership[['cms_certification_number_(ccn)', 'owner_type']].drop_duplicates()
print(f"\nUnique ownership records: {len(ownership_clean)}")

# Create final facility archetypes dataframe
facility_archetypes = contract_metrics.copy()
facility_archetypes['ccn'] = facility_archetypes['provnum']

facility_archetypes = pd.merge(
    facility_archetypes,
    ownership_clean,
    left_on='ccn',
    right_on='cms_certification_number_(ccn)',
    how='left'
)

print("\nFinal Archetypes Shape:", facility_archetypes.shape)

# Now print the analysis
print("\n" + "="*50)
print("FACILITY ARCHETYPE ANALYSIS")
print("="*50)

# Size-based analysis
print("\nA. Size-Based Characteristics:")
size_summary = facility_archetypes.groupby('size_category', observed=True).agg({
    'rn_contract_ratio': ['mean', 'std', 'count']
}).round(3)
print(size_summary)

# Contract usage analysis
print("\nB. Contract Usage Characteristics:")
contract_summary = facility_archetypes.groupby('contract_category', observed=True).agg({
    'rn_contract_ratio': ['mean', 'std', 'count']
}).round(3)
print(contract_summary)

# Ownership analysis
print("\nC. Ownership Type Characteristics:")
ownership_summary = facility_archetypes.groupby('owner_type', observed=True).agg({
    'rn_contract_ratio': ['mean', 'std', 'count']
}).round(3)
print(ownership_summary)

# Distribution analysis
print("\nDistribution of Facilities:")
print("\nBy Size:")
print(facility_archetypes['size_category'].value_counts(normalize=True).round(3) * 100)

print("\nBy Contract Usage:")
print(facility_archetypes['contract_category'].value_counts(normalize=True).round(3) * 100)

print("\nBy Ownership:")
print(facility_archetypes['owner_type'].value_counts(normalize=True).round(3) * 100)

# Key archetype analysis
print("\nKey Archetype Details:")
small_individual = facility_archetypes[
    (facility_archetypes['size_category'] == 'Small') & 
    (facility_archetypes['owner_type'] == 'Individual')
]

large_high_contract = facility_archetypes[
    (facility_archetypes['size_category'] == 'Large') & 
    (facility_archetypes['contract_category'] == 'High Contract')
]

print("\nSmall Individual-Owned Facilities:")
print(f"Count: {len(small_individual)}")
if len(small_individual) > 0:
    print(f"Average RN Contract Ratio: {small_individual['rn_contract_ratio'].mean():.3f}")
    print(f"Median Census: {small_individual['mdscensus'].median():.1f}")

print("\nLarge High-Contract Facilities:")
print(f"Count: {len(large_high_contract)}")
if len(large_high_contract) > 0:
    print(f"Average RN Contract Ratio: {large_high_contract['rn_contract_ratio'].mean():.3f}")
    print(f"Median Census: {large_high_contract['mdscensus'].median():.1f}")

# Calculate correlations if we have the data
correlation = facility_archetypes['mdscensus'].corr(facility_archetypes['rn_contract_ratio'])
print(f"\nCorrelation between Facility Size and RN Contract Ratio: {correlation:.3f}")


Facility Size Stats:
Number of facilities: 14564
Census range: 1.4 to 732.3

Facility Size Categories (Quartile Based):
mdscensus
Small         3641
Medium        3641
Large         3641
Very-Large    3641
Name: count, dtype: int64

Contract Metrics Stats:
Number of facilities with contract metrics: 14564
RN contract ratio range: 0.000 to 1.000

Unique ownership records: 26339

Final Archetypes Shape: (25968, 10)

FACILITY ARCHETYPE ANALYSIS

A. Size-Based Characteristics:
              rn_contract_ratio             
                           mean    std count
size_category                               
Small                     0.077  0.172  6238
Medium                    0.063  0.146  6533
Large                     0.065  0.149  6679
Very-Large                0.079  0.159  6518

B. Contract Usage Characteristics:
                  rn_contract_ratio             
                               mean    std count
contract_category                               
Low Contract           